In [137]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.integrate import solve_ivp
from scipy.optimize import minimize

In [138]:
csv_url="https://raw.githubusercontent.com/glouppe/proj0016-epidemic-data/main/data.csv"
df = pd.read_csv(csv_url)

In [142]:
df.head(30)

,Day,num_positive,num_tested,num_hospitalised,num_cumulative_hospitalizations,num_critical,num_fatalities
0,1,0,0,1,1,0,0
1,2,5,8,1,1,0,0
2,3,10,12,2,2,0,0
3,4,11,16,2,2,0,0
4,5,9,12,2,2,0,0
5,6,9,10,2,2,0,0
6,7,7,9,2,2,0,0
7,8,18,22,4,4,0,0
8,9,16,23,5,6,0,0
9,10,16,21,7,8,0,0


In [144]:
nmax=max(df.index)

## model definition ##

$$dS=-\theta_{0}\frac{S(I+C)}{(N-D)}$$
$$dE=\theta_{0}\frac{S(I+C)}{(N-D)} -\theta_{1}E$$
$$dI=0.6 \times \theta_{1}E-(\theta_{2}+\theta_{6})I$$
$$dC=0.4 \times \theta_{1}E-\theta_{6}C$$
$$dR=(\theta_{6}C+\theta_{6})I+\theta_{7}H$$
$$dH=\theta_{2}I-(\theta_{3}+\theta_{7})H+\theta_{4}ICU$$
$$dICU= \theta_{3}H-\theta_{4}ICU-\theta_{5}D$$
$$dD=\theta_{3}H$$



#### predictions from model to num_... ####
$$positive=\frac{S_{e}dI}{2}+dH$$
$$test=\frac{dI}{2}+dH$$
$$hospitalized=H$$
$$critical=ICU$$
$$fatalities=D$$

In [86]:
def predict_obs(array):
    df=pd.DataFrame(array.transpose())
    df2=pd.DataFrame()
    df2['num_positive']=0.425*df[2].diff()+df[5].diff()
    df2['num_tested']=0.425*df[2].diff()+df[5].diff()
    df2['num_hospitalized']=df[5]
    df2['num_critical']=df[6]
    df2['num_fatalities']=df[7]
    return df2
    

In [116]:
def objective_value(odedf,data):
    difference=data.drop(columns=['Day','num_cumulative_hospitalizations'])-odedf
    sum=difference.pow(2).sum().sum()
    return sum
    

In [154]:

def sumsq(theta):
    def model(t,y):
        S=y[0]
        E=y[1]
        I=y[2]
        C=y[3]
        R=y[4]
        H=y[5]
        ICU=y[6]
        D=y[7]
        dS=-theta[0]*(S*(I+C))/(100000-D)
        dE=theta[0]*(S*(I+C))/(100000-D)-theta[1]*E
        dI=0.6*theta[1]*E-(theta[2]+theta[6])*I
        dC=0.4*theta[1]*E-theta[6]*C
        dR=theta[6]*(C+I)+theta[7]*H
        dH=theta[2]*I+theta[4]*ICU-(theta[3]+theta[7])*H
        dICU=theta[3]*H-theta[4]*ICU-theta[5]*ICU
        dD=theta[5]*ICU
        return [dS,dE,dI,dC,dR,dH,dICU,dD]
    sol = solve_ivp(model,[0,nmax],[100000-theta[8]-theta[9],theta[8],theta[9],0,0,0,0],t_eval=np.arange(0,nmax,1))
    df2=predict_obs(sol.y)
    sum=objective_value(df2,df)
    return(sum)

In [155]:
msol = minimize(sumsq,[1 for i in range (10)],method='BFGS')


IndexError: index 7 is out of bounds for axis 0 with size 7

In [147]:
msol.x
print("an approximation of Ro is:",msol.x[0]/msol.x[6])

an approximation of Ro is: 0.2981629330027452


## fit graph ##

In [148]:
theta=msol.x
def model(t,y):
        S=y[0]
        E=y[1]
        I=y[2]
        C=y[3]
        R=y[4]
        H=y[5]
        ICU=y[6]
        D=y[7]
        dS=-theta[0]*(S*(I+C))/(100000-D)
        dE=theta[0]*(S*(I+C))/(100000-D)-theta[1]*E
        dI=0.6*theta[1]*E-(theta[2]+theta[6])*I
        dC=0.4*theta[1]*E-theta[6]*C
        dR=theta[6]*(C+I)+theta[7]*H
        dH=theta[2]*I+theta[4]*ICU-(theta[3]+theta[7])*H
        dICU=theta[3]*H-theta[4]*ICU-theta[5]*ICU
        dD=theta[5]*ICU
        return [dS,dE,dI,dC,dR,dH,dICU,dD]
sol = solve_ivp(model,[0,nmax],theta,t_eval=np.arange(0,nmax,1))

In [149]:
predictions=predict_obs(sol.y)

In [150]:
predictions

,num_positive,num_tested,num_hospitalized,num_critical,num_fatalities
0,NaN,NaN,0.646530,5.522327,0.256326
1,1.711130,1.711130,0.973384,2.773179,2.839491
2,-0.421317,-0.421317,0.626017,1.386845,4.133951
3,-0.112754,-0.112754,0.521168,0.679838,4.776693
4,-0.075413,-0.075413,0.446443,0.318979,5.086525
5,-0.064394,-0.064394,0.382108,0.136599,5.227066
6,-0.055838,-0.055838,0.326275,0.046180,5.282689
7,-0.048126,-0.048126,0.278150,0.002897,5.296786
8,-0.041264,-0.041264,0.236887,-0.016442,5.291517
9,-0.035265,-0.035265,0.201621,-0.023807,5.278066


In [ ]:
sol = solve_ivp(model,[0,14],[99,1,0,0,0,0,0],t_eval=np.arange(0,14,1))